# Scraping Spotifiy playlist id

In [1]:
import os
import pandas as pd
import numpy as np
import json
import random, string
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline

In [2]:
# Settings for pandas
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Settings for seaborn
sns.set(rc = {'figure.figsize':(20,8)})

In [3]:
import warnings  # To ignore warnings

def warn(*args, **kwargs):
    pass

warnings.warn = warn

## Gathering ids using spotipy

[spotipy reference](https://spotipy.readthedocs.io/en/2.16.1/#features)

In [4]:
# import necessary libaries
import spotipy
import spotipy.util as util
import requests
import shutil
import urllib.request


# A list to store id and its emotion of playlists
row_emotion_id_list = []

# The dictionary's keys are the emotions, and their values are lists containing the terms used to search for the playlists
emotion_dict = {"happy": ["happy", "happy vibe", "good mood", "행복", "즐거움", "신남"], 
                "sad": ["sad","depressed","lonely", "슬픔","우울", "외로움"], 
                "relaxed": ["relax","chill","calm", "진정","잔잔"], 
                "angry": ["angry","pissed","annoyed","rage","짜증"]}  


# setup authorization(more information on this in the setup authorization file)
username = 'jaeyonggy'
client_id = 'c28145e0e2be4ba5b14b480036c26782'
client_secret = '1aee1644d4de44f0a752e3e41a2e2cc1'
redirect_uri = 'http://localhost:7778/callback'
scope = 'user-library-read'


token = util.prompt_for_user_token(username=username, client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri, scope=scope)

spotify = spotipy.Spotify(auth=token)


# loop for each emotion_dict keys

for emotion in tqdm(emotion_dict.keys()):
    no_duplicates = []  # a list to contain unique ids
    
    for emotion_items in emotion_dict[emotion]:  # emotion_items are going to be used to search for playlists
        ids = spotify.search(emotion_items, type='playlist', limit=50)  # 50 is the max unfortunately
        
        for item in ids['playlists']['items']:
            if item['id'] not in no_duplicates:  # To filter out duplicate ids
                no_duplicates.append(item['id'])
                row_emotion_id_list.append([emotion, item['id'], item['name']])

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.11s/it]


In [5]:
emotion_id_df = pd.DataFrame(row_emotion_id_list, columns=['emotion','id','playlist name'])
# emotion_id_df.to_csv('emotion_id.csv', index=False, encoding='utf-8-sig')
emotion_id_df

,emotion,id,playlist name
0,happy,0ErX7GQiwz8nxRMmUBFLJn,Happy
1,happy,37i9dQZF1DXdPec7aLTmlC,Happy Hits!
2,happy,0LvrtAGoMBtZdNdHCRtMWf,Happy new year 🎈🎄
3,happy,37i9dQZF1DWTwbZHrJRIgD,Happy Weekend
4,happy,37i9dQZF1DWSf2RDTDayIx,Happy Beats
...,...,...,...
996,angry,54VgzaNfaOe1C7TpT0CKmc,퇴근길 만원버스 짜증방지용
997,angry,7B0SkSpz5yJ1X0nilufCa7,짜증나
998,angry,55HoLLE0ccHLumPMtOetXb,짜증나
999,angry,56PAwSWpF9INIUaq1uv0OW,K-Rap 짜증나


In [6]:
happy_id_df = emotion_id_df[emotion_id_df['emotion']=='happy']
happy_id_df.to_csv('happy_id.csv', index=False, encoding='utf-8-sig')
happy_id_df

,emotion,id,playlist name
0,happy,0ErX7GQiwz8nxRMmUBFLJn,Happy
1,happy,37i9dQZF1DXdPec7aLTmlC,Happy Hits!
2,happy,0LvrtAGoMBtZdNdHCRtMWf,Happy new year 🎈🎄
3,happy,37i9dQZF1DWTwbZHrJRIgD,Happy Weekend
4,happy,37i9dQZF1DWSf2RDTDayIx,Happy Beats
...,...,...,...
246,happy,43fwYOi2itjIsE4VaIGGjD,신남
247,happy,70kLOLq4gYUClrc9Wti76o,신남
248,happy,5SwZ8PQpED3Ls7UrmOUiYJ,신남
249,happy,0bQNYRQUhVhxougBsGSXeR,신남


In [7]:
sad_id_df = emotion_id_df[emotion_id_df['emotion']=='sad']
sad_id_df.to_csv('sad_id.csv', index=False, encoding='utf-8-sig')
sad_id_df

,emotion,id,playlist name
251,sad,5i1zqVCuIuQKYpPznAgEMs,Sad
252,sad,6nxPNnmSE0d5WlplUsa5L3,Sad Songs 🥺
253,sad,3c0Nv5CY6TIaRszlTZbUFk,sad songs 2022 😢 crying and depressing music
254,sad,37i9dQZF1DWZqdNJSufucb,sad 😔
255,sad,37i9dQZF1DWSqBruwoIXkA,sad hour
...,...,...,...
526,sad,7aVI4cFLgdr3AnJcJ3cNLC,외로움
527,sad,2XyVRVR8PaKMMVJQ0mpAV3,외로움
528,sad,6jT5t9ptPca4AJA8bI6F6N,외로움
529,sad,5TutrxispxXFs2jRiKDtEq,나의 외로움이 너를 부를 때


In [8]:
relaxed_id_df = emotion_id_df[emotion_id_df['emotion']=='relaxed']
relaxed_id_df.to_csv('relaxed_id.csv', index=False, encoding='utf-8-sig')
relaxed_id_df

,emotion,id,playlist name
531,relaxed,3MAs8P5Vpx6TMIVlvUfDv2,Relax
532,relaxed,37i9dQZF1DX3Ogo9pFvBkY,Ambient Relaxation
533,relaxed,1r4hnyOWexSvylLokn2hUa,Relaxing Music
534,relaxed,0n81ha8dSdYLDVc8VpCPsd,Relaxing Music 2021 🍃 Relaxing Piano Songs to ...
535,relaxed,37i9dQZF1DXebxttQCq0zA,Relaxing Massage
...,...,...,...
772,relaxed,7xj63vruUa6njNnOuMfPbv,잔잔바리
773,relaxed,0jlTWHJzxb1DWc7cHUaV2l,잔잔한 카톨릭 Hymns
774,relaxed,4a8H4nMhc14S5hr5bE1Kes,잔잔
775,relaxed,65J3EA2dxpVUW5atKG45Bi,잔잔


In [9]:
angry_id_df = emotion_id_df[emotion_id_df['emotion']=='angry']
angry_id_df.to_csv('angry_id.csv', index=False, encoding='utf-8-sig')
angry_id_df

,emotion,id,playlist name
777,angry,457PRAhJMUXaS8s6Evh6Hd,ANGRY ASS MUSIC FOR LIFTING HEAVY ASS WEIGHTS 😈
778,angry,5bWn8XcgI3hv2vpWjNq8la,angry songs :)
779,angry,5O12S9z3O8dEhHWt3bPbxm,for when you’re angry
780,angry,4LkEuyJjv4MVIDWh9QExsp,ANGRY BREAKUP PLAYLIST
781,angry,15Glcc8LdAloMFFfbODCl7,Angry Rock
...,...,...,...
996,angry,54VgzaNfaOe1C7TpT0CKmc,퇴근길 만원버스 짜증방지용
997,angry,7B0SkSpz5yJ1X0nilufCa7,짜증나
998,angry,55HoLLE0ccHLumPMtOetXb,짜증나
999,angry,56PAwSWpF9INIUaq1uv0OW,K-Rap 짜증나
